**Import Library**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
import sympy as sp
from PIL import Image, ImageOps
import os

**Size of Data**

In [2]:
#Size of the image
Face = Image.open("archive\\s1\\1.pgm") 
FaceArray = im.pil_to_array(Face)
Height, Width = FaceArray.shape
#Number of classes
num_class = sum(1 for entry in os.scandir("archive") if not entry.is_file())
#Number of files in a class
num_files = sum(1 for entry in os.scandir("archive\\s1") if entry.is_file())
#Total number of files
num_total = num_files*num_class

**Function**

In [3]:
#Process the Image into a flat 1D array
def ProcessImage(img):
    Grayscale = img.convert('L')
    Facematrix = im.pil_to_array(Grayscale)
    FlatFaceArray = np.concat(Facematrix,axis=None)/255

    return FlatFaceArray

#Make the Data Matrix
def Datamatrix(clsdir: str, num_files: int):
    for i in range(num_files):
        imgdir = clsdir +"\\" + str(i+1) + ".pgm"
    #Import image
        Face = Image.open(imgdir)
        FlatFaceArray = ProcessImage(Face)
    #Combine each 1D array in to a matrix
        if i == 0:
            FaceMatrix = FlatFaceArray
        else:
            FaceMatrix = np.column_stack((FaceMatrix,FlatFaceArray))
    return FaceMatrix

#Make a Data Matrix for a class k
def ClassMatrix(k: int):
    clsdir = "archive\\s" + str(k)
    return Datamatrix(clsdir, num_files)
ClassMatrix(2).shape



(10304, 10)

**Make the overall Data Matrix**

In [4]:

for i in range(num_class):
    if i == 0:
        FullMatrix = ClassMatrix(i+1)
    else:
        FullMatrix = np.append(FullMatrix, ClassMatrix(i+1),axis=1)
FullMatrix

array([[0.18823529, 0.23529412, 0.15294118, ..., 0.49019608, 0.46666667,
        0.49019608],
       [0.19215686, 0.23529412, 0.17254902, ..., 0.46666667, 0.47058824,
        0.48627451],
       [0.17647059, 0.24313725, 0.20784314, ..., 0.48627451, 0.47058824,
        0.48627451],
       ...,
       [0.18431373, 0.1254902 , 0.11372549, ..., 0.14117647, 0.34901961,
        0.14117647],
       [0.18039216, 0.13333333, 0.10196078, ..., 0.15294118, 0.36862745,
        0.1372549 ],
       [0.18039216, 0.13333333, 0.11372549, ..., 0.15686275, 0.33333333,
        0.13333333]])

In [5]:
#The average face
MeanFace=np.mean(FullMatrix, axis=1)
ColumnMeanFace= MeanFace[:, np.newaxis]
print(MeanFace)


[0.3357549  0.33559804 0.33696078 ... 0.30145098 0.2975098  0.2950098 ]


**Calculate the eigenvalues and eigenfaces**

In [99]:
#Minus the mean
CenteredFullMatrix=FullMatrix  - ColumnMeanFace
#Find the Eigenface of the Data
BtB=CenteredFullMatrix.T @ CenteredFullMatrix
V, S, Vt = np.linalg.svd(BtB)
#Find the needed amount of eigenvalues
eigsum=np.sum(S)
csum=0
for i in range(S.shape[0]):
    csum +=S[i]
    if csum > 0.95*eigsum:
        e90=i
        break

#S_reduced=S[0:e90]
#S_matrix = np.zeros(np.diag(S).shape)

#S_matrix[:e90, :e90] = np.sqrt(np.diag(S_reduced))
S_matrix = np.sqrt(np.diag(S))
S_inv=np.linalg.inv(S_matrix)
U=CenteredFullMatrix @ V @ S_inv
#Reducing the eigenvalues and vector to the needed amount
U_reduced = U[:,:e90]
S_reduced= np.sqrt(np.diag(S[0:e90]))
U 

array([[ 2.12507923e-03,  1.46851506e-02,  1.99294881e-02, ...,
         7.24583408e-03,  1.97647330e-03,  1.88152095e-09],
       [ 2.11276614e-03,  1.46139383e-02,  2.00092007e-02, ...,
         5.62517084e-04, -2.45204094e-03,  9.40760474e-09],
       [ 2.14250419e-03,  1.46318643e-02,  1.98385174e-02, ...,
        -2.75106052e-03,  3.55081707e-04, -1.74040688e-09],
       ...,
       [ 7.04005567e-03, -1.05610380e-02,  1.41636829e-02, ...,
         4.57939269e-03,  9.57219506e-03, -7.14977960e-09],
       [ 6.39096175e-03, -9.70069545e-03,  1.43943595e-02, ...,
         1.45672324e-04,  9.97750968e-03, -1.59929281e-08],
       [ 7.34479428e-03, -8.81892481e-03,  1.48748947e-02, ...,
         2.13634292e-02,  1.05696646e-02, -1.09128215e-08]])

In [100]:
#Reconstruct an example Eigenface as a picture
Eigenface=U_reduced 
face = Eigenface[:,1]
def ReconstructImage(face):
    face = face*255
    face = np.split(face, Height)
    for i in range(len(face)):
        if i == 0:
            recface=face[0]
        else:
            recface = np.vstack((recface,face[i]))
    recim = Image.fromarray(recface).convert("L")
    recim.save('test.png','PNG')

In [101]:
#Projecting a vector in to the facespace
def Facespace(vector):
    Omega = np.array([])
    for i in range(e90):
        component=U_reduced[:,i] @ vector
        Omega = np.append(Omega,[[component]] )
    return Omega
#Omega= Omega[:, np.newaxis]

#the average of a class k in facespace
def avgClassOmega(k: int):
    S = 0
    for i in range(num_files):
        S+= Facespace(ClassMatrix(k)[:,i])
    print(S)
    return S/len(range(num_files))

#Find the threshold for the picture to be indentified as a face in class k
def Classepsilon(k: int):
    Cepsilon = 0
    for i in range(num_files):
        e=np.linalg.norm(Facespace(ClassMatrix(k)[:,i])-avgClassOmega(k))
        if e > Cepsilon:
            Cepsilon = e
    return Cepsilon*1.2

np.linalg.norm(avgClassOmega(34)), np.linalg.norm(Facespace(ClassMatrix(1)[:,1])-avgClassOmega(34))


[-2.73648789e+02  1.46603398e+02 -3.51737226e+00 -4.41793637e+01
  1.40568979e+02  9.68929610e+01  6.84290831e+01 -3.15371240e+01
 -1.05244235e+02  2.21012483e+01  7.91980893e+01  1.77538447e+01
 -5.12287164e+01  1.83641236e+01 -2.20737139e+01 -1.89415367e+01
 -2.30999310e+01 -3.64950306e+00 -5.29395051e+00 -3.76904838e+00
  8.01300895e+00 -2.80030037e+01 -7.51249513e+00  3.09527054e+01
 -6.42469751e+00  3.32126945e+00 -2.26609025e+01  1.14572267e+01
 -1.29928032e+00  2.61820114e+01 -3.76855009e+00  1.23315852e+00
  1.03140649e+01  5.68374748e-01 -2.28533531e+00  3.60679715e+00
 -9.29181555e+00 -6.30767008e+00 -8.70125061e+00  2.15026772e+01
 -5.90406464e+00  5.60962979e+00  8.54574990e+00  1.57872646e+01
 -5.91623639e+00  3.21852060e+01  7.34631185e+00 -1.13594034e+01
 -1.73744754e+01  1.16950348e+01  2.04316664e+00 -3.90217878e+00
  4.05537061e+00 -4.99953499e+00  1.03351960e+01  3.91893078e+00
 -1.07797536e+00 -1.96167843e+01  7.30043316e+00  3.67324263e-01
 -5.23256883e+00 -7.97832

(np.float64(40.647275248958145), np.float64(25.909281410455716))

In [102]:

#The average of all the data in facespace
#S = 0
#for i in range(num_total):
#    S = S+ Facespace(CenteredFullMatrix[:,i])
#avgOmega = S#/len(range(num_files))
#Find the threshold for the picture to be identified as a face, let that be epsilon
epsilon=0
#Array of a facevector in face space
def FacespaceArray(facearray):
    return Facespace(facearray) @ U_reduced.T

for i in range(num_total):
    facearray = CenteredFullMatrix[:,i]
    e = np.linalg.norm(facearray-FacespaceArray(facearray))
    if e > epsilon:
        epsilon = e
#Let the threshold be 2epsilon
epsilon = 2*epsilon
epsilon
#Facespace(ClassMatrix(2)[:,1]).shape

np.float64(8.54692502122684)

**Face recognization**

In [104]:
#ball = np.array([])
#for i in range(num_class):
#    ball = np.append(ball,Classepsilon(i+1))
#ball
# 
Facespace(CenteredFullMatrix[:,1]).shape


(189,)

In [105]:

#Input = ProcessImage(Image.open("32.JPG"))-MeanFace
#Input.shape
#Let the image that need to be specify is input
def Classify(imgdir: str ):
    image=Image.open(imgdir)
    facearray = ProcessImage(image)-MeanFace
    dist = np.linalg.norm(facearray - FacespaceArray(facearray))
    if epsilon > dist:
        print("this image contains a face")
    else: 
        print("this image does not contain a face")
    print(dist)


In [106]:
Height, Width

(112, 92)

In [ ]:
Classify("18.JPG")
#troll

this image does not contain a face
8.777071572728351


In [124]:
ReconstructImage(FacespaceArray(ProcessImage(Image.open("18.JPG"))))

In [120]:
ReconstructImage(FacespaceArray(CenteredFullMatrix[:,4]))
Image.open("archive\\s1\\5.pgm").show()